# Import Packages

In [1]:

# sfsdds
import sys

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


pd.set_option('display.max_rows', None)


from pydfs_lineup_optimizer import Player,get_optimizer, Site, Sport, CSVLineupExporter, RandomFantasyPointsStrategy, AfterEachExposureStrategy, TeamStack, PlayersGroup, Stack
import re



# Setup Optimizer

In [10]:
optimizer = get_optimizer(Site.DRAFTKINGS, Sport.FOOTBALL)
#optimizer = get_optimizer(Site.DRAFTKINGS_CAPTAIN_MODE, Sport.FOOTBALL)
#optimizer.load_players_from_csv('DKSalaries_20251117_early.csv')
optimizer.load_players_from_csv('DKSalaries_20251121_late.csv')

In [28]:
pp_list = [
    'Markkanen', 'Sharpe', 'Giddey', 'Okongwu', 'Clingan', 'Grant',
    'Avdija', 'George', 'Gillespie', 'Silva', 'Black', 'Goodwin',
    'Brooks', 'Queen', 'Johnson', 'Buzelis', 'Camara', 'Sheppard',
    'Gafford', 'Sengun'
]

# Assuming 'optimizer' and 'optimizer.player_pool' are already defined and populated.

# Define the new deviation values you want to apply to these players
new_min_deviation = 0.3  # Example value
new_max_deviation = 0.6  # Example value

# Loop through the list of player names and update their deviations
for player_name in pp_list:
    # Retrieve the player object from the optimizer's player pool
    player = optimizer.player_pool.get_player_by_name(player_name)
    
    # Check if the player was found before trying to set attributes
    if player:
        player.min_deviation = new_min_deviation
        player.max_deviation = new_max_deviation
        # Optional: you could print a confirmation
        # print(f"Updated deviation for {player_name}")
    else:
        # Handle cases where a name might not be in the pool
        print(f"Warning: Player '{player_name}' not found in player pool.")

# The rest of your existing code still applies globally or to specific individuals as needed:
optimizer.set_fantasy_points_strategy(RandomFantasyPointsStrategy(max_deviation=0.2))

# You can still set specific exceptions for other players outside the list:
#harden = optimizer.player_pool.get_player_by_name('Harden')
#if harden:
#    harden.min_deviation = 0.3
#    harden.max_deviation = 0.6

#westbrook = optimizer.player_pool.get_player_by_name('Westbrook')
#if westbrook:
#    westbrook.min_deviation = 0
#    westbrook.max_deviation = 0


# Run/Process Results

In [11]:
has_results =False

for lineup in optimizer.optimize(n=3):#, max_exposure=0.5):
    print(lineup)                         # existing printout

    if has_results:
        eval_df, totals = evaluate_lineup(lineup, results_lookup)
        #display(eval_df)                  # per-player view with proj / actual / %Drafted
        #print("Projected FP:", totals["Proj_Total"])
        print("Actual FP:   ", totals["Actual_Total"])
        print("%Own:", totals["%Own"])
        print('\n\n\n')


 1. QB      Patrick Mahomes               QB    KC             CAR@KC   23.876         7500.0$   
 2. RB      Chuba Hubbard                 RB    CAR            CAR@KC   14.469         5500.0$   
 3. RB      Travis Etienne Jr.            RB    JAX            IND@JAX  15.113         7600.0$   
 4. WR      Christian Watson              WR    GB             WAS@GB   11.992         5200.0$   
 5. WR      Matthew Golden                WR    GB             WAS@GB   12.255         6100.0$   
 6. WR      Terry McLaurin                WR    WAS            WAS@GB   12.745         7000.0$   
 7. TE      Tucker Kraft                  TE    GB             WAS@GB   9.947          5200.0$   
 8. FLEX    Jacory Croskey-Merritt        RB    WAS            WAS@GB   14.987         3000.0$   
 9. DST     Jaguars                       DST   JAX            IND@JAX  2.714          2900.0$   

Fantasy Points 118.10
Salary 50000.00

 1. QB      Jayden Daniels                QB    WAS            WAS@GB   23.941

In [11]:
# player list

pp_list = ['Markkanen',       
  'Sharpe',      
     'Giddey',     
  'Okongwu',      
 'Clingan',      
    'Grant',    
     'Avdija',    
  'George',   
'Gillespie',    
'Silva',    
   'Black',    
  'Goodwin',    
   'Brooks',    
     'Queen',    
   'Johnson',    
  'Buzelis',    
  'Camara',      
   'Sheppard',    
  'Gafford',    
  'Sengun']    

In [12]:
# Optimizer settings

# number of lineups
n = 10 

# max repeating player
max_repeat= 5


# minimum salary for lineup
optimizer.set_min_salary_cap(49000)


# max/min teams
min_teams=4
max_teams=6

# team exposures
#optimizer.set_teams_max_exposures(0.5)  # Set 0.5 exposure for all teams
#optimizer.set_teams_max_exposures(0.5, exposures_by_team={'NYY': 0.8})  # Set 0.5 exposure for all teams except NYY and 0.8 exposure for NYY
#optimizer.set_teams_max_exposures(exposures_by_team={'NYY': 0.8})  # Set 0.5 exposure only for NYY
#optimizer.set_teams_max_exposures(exposures_by_team={'NYY': 0.5, 'NYM': 0.5}, exposure_strategy=AfterEachExposureStrategy)  # Use another exposure strategy



# STACKING


# team stacking

optimizer.add_stack(TeamStack(3))  # stack 3 players
optimizer.add_stack(TeamStack(3, for_teams=['NE', 'BAL', 'KC']))  # stack 3 players from any of specified teams
optimizer.add_stack(TeamStack(3, for_positions=['QB', 'WR', 'TE']))  # stack 3 players with any of specified positions
optimizer.add_stack(TeamStack(3, spacing=2))  # stack 3 players close to each other in range of 2 spots.
optimizer.add_stack(TeamStack(3, max_exposure=0.5))  # stack 3 players from same team with 0.5 exposure for all team stacks
optimizer.add_stack(TeamStack(3, max_exposure=0.5, max_exposure_per_team={'MIA': 0.6}))  # stack 3 players from same team with 0.5 exposure for all team stacks and 0.6 exposure for MIA


# Position Stack

optimizer.add_stack(GameStack(3))  # stack 3 players from the same game
optimizer.add_stack(GameStack(5, min_from_team=2))  # stack 5 players from the same game, 3 from one team and 2 from another



# custom group stack
rodgers_adams_group = PlayersGroup(optimizer.player_pool.get_players('Aaron Rodgers', 'Davante Adams'), max_exposure=0.5)
brees_thomas_group = PlayersGroup(optimizer.player_pool.get_players('Drew Brees', 'Michael Thomas'), max_exposure=0.5)
optimizer.add_stack(Stack([rodgers_adams_group, brees_thomas_group]))



# additional stack




NameError: name 'TeamStack' is not defined

In [ ]:


optimizer.set_total_teams(min_teams=min_teams, max_teams=max_teams)
optimizer.set_max_repeating_players(max_repeat)
lineups = list(optimizer.optimize(n))
optimizer.export('result.csv')

In [ ]:
lu = pd.read_csv('result.csv', index_col = False)
lu.head()

,PG,SG,SF,PF,C,G,F,UTIL,Budget,FPPG
0,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Andre Drummond(614746),Tyrese Maxey(1186160),Kyle Filipowski(1403609),Jusuf Nurkic(830642),50000.0,270.74
1,Keyonte George(1408151),Ace Bailey(1519363),Michael Porter Jr.(1061724),Collin Murray-Boyles(1465110),Andre Drummond(614746),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),OG Anunoby(900535),50000.0,270.01
2,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Jusuf Nurkic(830642),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Neemias Queta(1142902),50000.0,269.31
3,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Andre Drummond(614746),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Santi Aldama(1177024),50000.0,269.25
4,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Jusuf Nurkic(830642),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Trendon Watford(1176529),50000.0,268.54


In [ ]:
lu

In [ ]:





# add columns to your lineup df
lu[["Actual_FP", "Total_Own"]] = lu.apply(lineup_results, axis=1)

lu.head()


,PG,SG,SF,PF,C,G,F,UTIL,Budget,FPPG,Actual_FP,Total_Own
0,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Andre Drummond(614746),Tyrese Maxey(1186160),Kyle Filipowski(1403609),Jusuf Nurkic(830642),50000.0,270.74,222.50,211.01
1,Keyonte George(1408151),Ace Bailey(1519363),Michael Porter Jr.(1061724),Collin Murray-Boyles(1465110),Andre Drummond(614746),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),OG Anunoby(900535),50000.0,270.01,253.00,211.33
2,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Jusuf Nurkic(830642),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Neemias Queta(1142902),50000.0,269.31,231.75,183.79
3,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Andre Drummond(614746),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Santi Aldama(1177024),50000.0,269.25,241.00,174.72
4,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Jusuf Nurkic(830642),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Trendon Watford(1176529),50000.0,268.54,216.25,195.42
